In [2]:
using LinearAlgebra, SparseArrays
using Symbolics

@variables λ e_1 f_1 e_2 f_2 e_3 f_3 e_4 f_4 e_5 f_5 f_6;

In [3]:
L =[
    1    0 0 0 0 0
    -e_1  1 0 0 0 0
    f_1 -e_2 1 0 0 0
    0   f_2 -e_3 1 0 0
    0   0 f_3 -e_4 1 0
    0   0 0  f_4 -e_5 1
    ]

println("L"); display(L)
D = diagm([f_1, f_2, f_3, f_4, f_5, f_6])^-1

L


6×6 Matrix{Num}:
 1     0     0     0     0     0
 -e_1  1     0     0     0     0
  f_1  -e_2  1     0     0     0
 0      f_2  -e_3  1     0     0
 0     0      f_3  -e_4  1     0
 0     0     0      f_4  -e_5  1

6×6 Matrix{Num}:
 1 / f_1     0        0        0        0        0
    0     1 / f_2     0        0        0        0
    0        0     1 / f_3     0        0        0
    0        0        0     1 / f_4     0        0
    0        0        0        0     1 / f_5     0
    0        0        0        0        0     1 / f_6

In [5]:
@variables θ_1 θ_2 θ_3 θ_4 θ_5 θ_6
@variables w_1 w_2 w_3 w_4 w_5 w_6
θ = [θ_1 θ_2 θ_3 θ_4 θ_5 θ_6]'
w = [w_1 w_2 w_3 w_4 w_5 w_6]'
[L * D * θ w]

6×2 Matrix{Num}:
 θ_1 / f_1                               w_1
     (-e_1*θ_1) / f_1 + θ_2 / f_2        w_2
     (-e_2*θ_2) / f_2 + θ_3 / f_3 + θ_1  w_3
     (-e_3*θ_3) / f_3 + θ_4 / f_4 + θ_2  w_4
 θ_5 / f_5 + (-e_4*θ_4) / f_4 + θ_3      w_5
     (-e_5*θ_5) / f_5 + θ_6 / f_6 + θ_4  w_6

θ1 = f1 * w1
θ2 = f2(w2 + θ1e1/f1)
θ3 = f3(w3 + θ2e2/f2 - θ1 )


In [5]:
A2 = L * D * L'

6×6 Matrix{Num}:
 1 / f_1         (-e_1) / f_1              …  0
   (-e_1) / f_1   (e_1^2) / f_1 + 1 / f_2     0
    1             -e_1 + (-e_2) / f_2         0
    0             1                           1
    0             0                           -e_4 + (-e_5) / f_5
    0             0                        …   f_4 + (e_5^2) / f_5 + 1 / f_6

## 先求出e, f, 然后根据w, 确定theta
```julia
let u_i = a_1 - e_i-1 # u_i 没必要存在

-e1/f1 = a1
e1 = a1 * f1
f2 = 1/(a0 - e1^2/f1)

-e2/f2 = e1 - a1
e2 = -f2(-a1 + e1) = f2(a1 - e1) = f2 * u2

f3 = 1/(a0 -e2^2/f2 - f1) (e2^2/f2 = u2 * e2,)

-e3/f3 = -a1 + e2 
e3 = f3 * (a1 - e2)
```

In [12]:
A = [
    2/3λ+6 λ/6-4 1 0 0 0
    λ/6-4 2λ/3+6 λ/6-4 1 0 0
    1 λ/6-4 2λ/3+6 λ/6-4 1 0
    0 1 λ/6-4 2λ/3+6 λ/6-4 1
    0 0 1 λ/6-4 2λ/3+6 λ/6-4
    0 0 0 1 λ/6-4 2λ/3+6
]

6×6 Matrix{Num}:
  6 + 2 / (3λ)  -4 + (1//6)*λ              1  …              0              0
 -4 + (1//6)*λ   6 + (2//3)*λ  -4 + (1//6)*λ                 0              0
             1  -4 + (1//6)*λ   6 + (2//3)*λ                 1              0
             0              1  -4 + (1//6)*λ     -4 + (1//6)*λ              1
             0              0              1      6 + (2//3)*λ  -4 + (1//6)*λ
             0              0              0  …  -4 + (1//6)*λ   6 + (2//3)*λ

## second step
$$
L^T c = \theta
$$

In [23]:
@variables c_1 c_2 c_3 c_4 c_5 c_6
c = [c_1 c_2 c_3 c_4 c_5 c_6]'

[L' * c θ]


6×2 Matrix{Num}:
 c_1 - c_2*e_1 + c_3*f_1  θ_1
 c_2 - c_3*e_2 + c_4*f_2  θ_2
 c_3 - c_4*e_3 + c_5*f_3  θ_3
 c_4 - c_5*e_4 + c_6*f_4  θ_4
 c_5 - c_6*e_5            θ_5
        c_6               θ_6